<a href="https://colab.research.google.com/github/Fish210/Alliance-Optimization-Model/blob/main/Alliance_Optimization_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#FTC Gradient Boosted (CatBoost) Alliance Opimization Model
#Developers: John Uy & Vishvak Gurram
#Date: February 1, 2026

##### Imports #####
from google.colab import files
uploaded = files.upload()  # upload your CSV
!pip -q install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import textwrap

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec

##### Load sheets & Compute points #####
FILE = "/content/FTC_ALLIANCE_ML_MODEL_TRAINING_LT2026(team_match_actions).csv"  # change if needed

df = pd.read_csv(FILE)
df = df[df["team"].notna()].copy()

count_cols = [
    "auto_leave",
    "auto_artifact_cl_count","auto_artifact_overflow_count","auto_motif_match_count",
    "tele_artifact_cl_count","tele_artifact_overflow_count","tele_motif_match_count","tele_depot_count",
]

for c in count_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

# Scoring constants (locked values for DECODE season)
df["auto_points"] = (
    3*df["auto_leave"]
    + 3*df["auto_artifact_cl_count"]
    + 1*df["auto_artifact_overflow_count"]
    + 2*df["auto_motif_match_count"]
)

df["tele_points"] = (
    3*df["tele_artifact_cl_count"]
    + 1*df["tele_artifact_overflow_count"]
    + 2*df["tele_motif_match_count"]
    + 1*df["tele_depot_count"]
)

df["total_points"] = df["auto_points"] + df["tele_points"]

# Dead match (strict: zero total points)
df["dead_match"] = (df["total_points"] == 0).astype(int)

TEAM_US = 3470

##### DARK MODE THEME + SAFE LAYOUT #####
DARK_BG = "#0b0f14"
CARD_BG = "#111827"
TEXT = "#e5e7eb"
MUTED = "#9ca3af"
GRID = "#1f2937"

mpl.rcParams.update({
    "figure.facecolor": DARK_BG,
    "savefig.facecolor": DARK_BG,
    "axes.facecolor": CARD_BG,
    "axes.edgecolor": GRID,
    "axes.labelcolor": TEXT,
    "xtick.color": MUTED,
    "ytick.color": MUTED,
    "text.color": TEXT,
    "axes.titlecolor": TEXT,
    "grid.color": GRID,
    "grid.alpha": 0.35,
    "font.size": 9,          # smaller so nothing cuts off
    "axes.titlesize": 11,
    "axes.labelsize": 9,
})

sns.set_theme(style="dark")

##### Compute team stats + FitScore vs Our Team #####
team_stats = df.groupby("team").agg(
    mean_total=("total_points","mean"),
    std_total=("total_points","std"),
    dead_rate=("dead_match","mean"),
    mean_auto=("auto_points","mean"),
    mean_tele=("tele_points","mean"),
    mean_auto_CL=("auto_artifact_cl_count","mean"),
    mean_tele_CL=("tele_artifact_cl_count","mean"),
    mean_auto_motif=("auto_motif_match_count","mean"),
    mean_tele_motif=("tele_motif_match_count","mean"),
    matches=("total_points","count"),
).reset_index()

team_stats["std_total"] = team_stats["std_total"].fillna(0)

# Fingerprint for redundancy penalty (scoring mix)
def fingerprint(row):
    auto = row["mean_auto"]
    tele = row["mean_tele"]
    motif = row["mean_auto_motif"] + row["mean_tele_motif"]
    cl = row["mean_auto_CL"] + row["mean_tele_CL"]
    vec = np.array([auto, tele, motif, cl], dtype=float)
    n = np.linalg.norm(vec)
    return vec / (n + 1e-9)

fps = {int(r["team"]): fingerprint(r) for _, r in team_stats.iterrows()}
us_vec = fps.get(int(TEAM_US), None)

us_mean = float(team_stats.loc[team_stats["team"]==TEAM_US, "mean_total"].iloc[0])
us_auto = float(team_stats.loc[team_stats["team"]==TEAM_US, "mean_auto"].iloc[0])
us_tele = float(team_stats.loc[team_stats["team"]==TEAM_US, "mean_tele"].iloc[0])

team_stats["expected_alliance_points"] = us_mean + team_stats["mean_total"]
team_stats["risk_penalty"] = 0.6*team_stats["std_total"] + 15*team_stats["dead_rate"]

def cosine_sim(a,b):
    denom = (np.linalg.norm(a)+1e-9)*(np.linalg.norm(b)+1e-9)
    return float(np.dot(a,b)/denom)

if us_vec is not None:
    team_stats["redundancy"] = team_stats["team"].apply(lambda t: cosine_sim(fps[int(t)], us_vec))
else:
    team_stats["redundancy"] = 0.0

# Coverage bonus (boost our weaker phase)
weaker = "auto" if us_auto < us_tele else "tele"
if weaker == "auto":
    team_stats["coverage_bonus"] = (team_stats["mean_auto"] - us_auto).clip(lower=0) * 0.15
else:
    team_stats["coverage_bonus"] = (team_stats["mean_tele"] - us_tele).clip(lower=0) * 0.15

team_stats["fit_raw"] = (
    team_stats["expected_alliance_points"]
    - team_stats["risk_penalty"]
    - 5.0*team_stats["redundancy"]
    + team_stats["coverage_bonus"]
)

mn, mx = team_stats["fit_raw"].min(), team_stats["fit_raw"].max()
team_stats["fit_score"] = 100*(team_stats["fit_raw"] - mn) / (mx - mn + 1e-9)

team_stats["fit_rank_vs_us"] = team_stats[team_stats["team"]!=TEAM_US]["fit_score"]\
    .rank(ascending=False, method="min")

##### Summaries + Game plan + Next-match prediction #####
def wrap_lines(lines, width=48):
    return [textwrap.fill(str(x), width=width) for x in lines]

def why_fit(row):
    reasons = []
    if row["mean_auto"] > us_auto + 1:
        reasons.append(f"Auto boost vs 3470 (+{row['mean_auto']-us_auto:.1f} pts)")
    if row["mean_tele"] > us_tele + 1:
        reasons.append(f"Tele boost vs 3470 (+{row['mean_tele']-us_tele:.1f} pts)")
    if row["dead_rate"] > 0:
        reasons.append(f"Reliability risk: dead_rate={row['dead_rate']:.0%}")
    if row["std_total"] > 8:
        reasons.append(f"High variance: STD={row['std_total']:.1f}")
    if row["redundancy"] > 0.85:
        reasons.append("Overlap risk: scoring mix similar to 3470")
    if not reasons:
        reasons.append("Balanced fit: solid output + manageable risk")
    return reasons[:5]

def game_plan(row):
    plan = []
    if row["mean_auto"] > us_auto and row["mean_tele"] <= us_tele:
        plan.append("Roles: Partner emphasizes Auto; 3470 anchors Tele throughput.")
    elif row["mean_tele"] > us_tele and row["mean_auto"] <= us_auto:
        plan.append("Roles: Partner emphasizes Tele; 3470 covers Auto consistency.")
    elif row["mean_auto"] > us_auto and row["mean_tele"] > us_tele:
        plan.append("Roles: Both strong—maximize throughput; avoid duplicating the same scoring lane.")
    else:
        plan.append("Roles: 3470 carries primary scoring; partner supports & stabilizes output.")

    motif = row["mean_auto_motif"] + row["mean_tele_motif"]
    cl = row["mean_auto_CL"] + row["mean_tele_CL"]
    if motif > cl:
        plan.append("Style: Motif-leaning—prioritize pattern opportunities when available.")
    else:
        plan.append("Style: CL-leaning—prioritize volume/throughput.")

    if row["dead_rate"] > 0:
        plan.append("Risk: Have a fallback plan if partner underperforms (dead match risk).")

    return plan[:4]

# Simple, stable next-match prediction (no ML training): 70% last match + 30% mean
def predict_next(team_id):
    sub = df[df["team"] == team_id].sort_values("match_id")
    if len(sub) == 0:
        return 0.0, 0.0, 0.0
    last_auto = float(sub["auto_points"].iloc[-1])
    last_tele = float(sub["tele_points"].iloc[-1])
    mean_auto = float(sub["auto_points"].mean())
    mean_tele = float(sub["tele_points"].mean())
    pred_auto = 0.7*last_auto + 0.3*mean_auto
    pred_tele = 0.7*last_tele + 0.3*mean_tele
    return pred_auto, pred_tele, pred_auto + pred_tele

def save_fig(pdf, fig):
    pdf.savefig(fig, bbox_inches="tight", pad_inches=0.35)
    plt.close(fig)

##### Generate the PDF dashboards (dark mode, no cutoff) #####
pdf_path = "FTC_Dashboards_Dark.pdf"

with PdfPages(pdf_path) as pdf:
    # --- Global summary pages ---
    hm = team_stats.copy().sort_values("fit_score", ascending=False)
    hm_cols = ["fit_score","expected_alliance_points","mean_total","std_total","dead_rate","mean_auto","mean_tele"]
    hm_show = hm[["team"] + hm_cols].set_index("team")

    # Heatmap (dark-friendly colormap)
    fig = plt.figure(figsize=(12.5, 6.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.heatmap(hm_show, annot=False, cmap="mako", ax=ax)
    ax.set_title("Teams vs 3470 — Fit + Key Metrics")
    save_fig(pdf, fig)

    # Top partners bar
    top = hm[hm["team"] != TEAM_US].head(10)
    fig = plt.figure(figsize=(12.5, 4.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.barplot(data=top, x="team", y="fit_score", ax=ax)
    ax.set_title("Top Partners for 3470 — Fit Score")
    ax.tick_params(axis="x", rotation=45)
    ax.set_xlabel("Team")
    ax.set_ylabel("Fit Score")
    ax.grid(True, axis="y")
    save_fig(pdf, fig)

    # Ceiling vs Risk scatter
    fig = plt.figure(figsize=(10.5, 5.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.scatterplot(data=hm, x="mean_total", y="std_total", size="matches", sizes=(40, 240), ax=ax, legend=False)
    ax.set_title("Ceiling vs Risk (Mean vs STD)")
    ax.set_xlabel("Mean Points")
    ax.set_ylabel("STD Points")
    ax.grid(True)
    save_fig(pdf, fig)

    # --- Per-team pages ---
    teams = sorted(df["team"].unique())
    for t in teams:
        row = team_stats[team_stats["team"] == t].iloc[0]
        pred_auto, pred_tele, pred_total = predict_next(t)

        # Data for line chart
        tdf = df[df["team"] == t].sort_values("match_id")
        usdf = df[df["team"] == TEAM_US].sort_values("match_id")

        # Page layout: GridSpec (prevents cutoffs)
        fig = plt.figure(figsize=(12.5, 7.2), constrained_layout=True)
        gs = GridSpec(2, 3, figure=fig, height_ratios=[1.05, 1.0], width_ratios=[1.15, 1.0, 1.05])

        # Line chart (top-left spanning 2 columns)
        ax1 = fig.add_subplot(gs[0, 0:2])
        ax1.plot(tdf["match_id"], tdf["total_points"], marker="o", linewidth=1.8, label=f"Team {int(t)}")
        ax1.plot(usdf["match_id"], usdf["total_points"], marker="o", linewidth=1.8, label=f"Team {TEAM_US}")
        ax1.set_title(f"Team {int(t)} vs {TEAM_US} — Real Points Over Matches")
        ax1.set_xlabel("Match")
        ax1.set_ylabel("Total Points")
        ax1.grid(True)
        ax1.legend(loc="upper left", fontsize=9)

        # Stats + prediction (top-right)
        ax2 = fig.add_subplot(gs[0, 2])
        ax2.axis("off")
        rank_val = "-" if t == TEAM_US else int(row["fit_rank_vs_us"])
        stats = [
            f"Fit Score: {row['fit_score']:.1f}",
            f"Fit Rank vs us: {rank_val}",
            f"Exp Alliance Pts: {row['expected_alliance_points']:.1f}",
            f"Mean/STD: {row['mean_total']:.1f} / {row['std_total']:.1f}",
            f"Dead Rate: {row['dead_rate']:.0%}",
            "",
            "Next-match prediction (stable):",
            f"Pred Auto: {pred_auto:.1f}",
            f"Pred Tele: {pred_tele:.1f}",
            f"Pred Total: {pred_total:.1f}",
        ]
        ax2.text(0.02, 0.98, "\n".join(stats), va="top", fontsize=10)

        # Auto/Tele bar chart (bottom-left)
        ax3 = fig.add_subplot(gs[1, 0])
        comp = pd.DataFrame({
            "metric":["Auto","Tele"],
            f"{int(t)}":[row["mean_auto"], row["mean_tele"]],
            f"{TEAM_US}":[us_auto, us_tele]
        }).set_index("metric")
        comp.plot(kind="bar", ax=ax3, rot=0)
        ax3.set_title("Avg Auto/Tele Points")
        ax3.grid(True, axis="y")
        ax3.legend(fontsize=8)

        # Motif/CL comparison (bottom-middle)
        ax4 = fig.add_subplot(gs[1, 1])
        row_us = team_stats[team_stats["team"] == TEAM_US].iloc[0]
        motif_team = row["mean_auto_motif"] + row["mean_tele_motif"]
        motif_us = row_us["mean_auto_motif"] + row_us["mean_tele_motif"]
        cl_team = row["mean_auto_CL"] + row["mean_tele_CL"]
        cl_us = row_us["mean_auto_CL"] + row_us["mean_tele_CL"]

        x = np.arange(2)
        ax4.bar(x - 0.18, [motif_team, cl_team], width=0.36, label=f"{int(t)}")
        ax4.bar(x + 0.18, [motif_us, cl_us], width=0.36, label=f"{TEAM_US}", alpha=0.65)
        ax4.set_xticks(x)
        ax4.set_xticklabels(["Motif", "CL"])
        ax4.set_title("Avg Motif / CL")
        ax4.grid(True, axis="y")
        ax4.legend(fontsize=8)

        # Summary + game plan (bottom-right)
        ax5 = fig.add_subplot(gs[1, 2])
        ax5.axis("off")

        reasons = wrap_lines(why_fit(row), width=46)
        plan = wrap_lines(game_plan(row), width=46)

        block = []
        block.append("Summary (why good/bad for us):")
        block += [f"- {r}" for r in reasons]
        block.append("")
        block.append("Game plan:")
        block += [f"- {p}" for p in plan]

        ax5.text(0.02, 0.98, "\n".join(block), va="top", fontsize=9)

        save_fig(pdf, fig)

print("Saved PDF:", pdf_path)

from google.colab import files
files.download(pdf_path)


Saving FTC_ALLIANCE_ML_MODEL_TRAINING_LT2026(team_match_actions).csv to FTC_ALLIANCE_ML_MODEL_TRAINING_LT2026(team_match_actions).csv
Saved PDF: FTC_Dashboards_Dark.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>